In [ ]:
import importlib
import json
import logging
import os
import shutil
import time
from datetime import datetime

from camel.agents import RolePlaying
from camel.configs import ChatGPTConfig
from camel.typing import TaskType, ModelType
from chatdev.chat_env import ChatEnv, ChatEnvConfig
from chatdev.statistics import get_info
from camel.web_spider import modal_trans
from chatdev.utils import log_visualize, now

class ChatChain:
  def __init__(self,
               config_path: str = None,
               config_phase_path: str = None,
               config_role_path: str = None,
               task_prompt: str = None,
               project_name: str = None,
               org_name: str = None,
               model_type: ModelType = ModelType.GPT_3_5_TURBO,
               code_path: str = None) -> Non:

        self.config_path = config_path
        self.config_phase_path = config_phase_path
        self.config_role_path = config_role_path
        self.project_name = project_name
        self.org_name = org_name
        self.model_type = model_type
        self.code_path = code_path

        with open(self.config_path, 'r', encoding="utf8") as file:
            self.config = json.load(file)
        with open(self.config_phase_path, 'r', encoding="utf8") as file:
            self.config_phase = json.load(file)
        with open(self.config_role_path, 'r', encoding="utf8") as file:
            self.config_role = json.load(file)

        # init chatchain config and recruitments
        self.chain = self.config["chain"]
        self.recruitments = self.config["recruitments"]
        self.web_spider = self.config["web_spider"]

        # init default max chat turn
        self.chat_turn_limit_default = 10
        #init ChatEnv
        self.chat_env_config = ChatEnvConfig(clear_structure=check_bool(self.config["clear_structure"]),
                                             gui_design=check_bool(self.config["gui_design"]),
                                             git_management=check_bool(self.config["git_management"]),
                                             incremental_develop=check_bool(self.config["incremental_develop"]),
                                             background_prompt=self.config["background_prompt"],
                                             with_memory=check_bool(self.config["with_memory"]))

        self.chat_env = ChatEnv(self.chat_env_config)

        # the user input prompt will be self-improved (if set "self_improve": "True" in ChatChainConfig.json)
        # the self-improvement is done in self.preprocess
        self.task_prompt_raw = task_prompt
        self.task_prompt = ""

        # init role prompts
        self.role_prompts = dict()
        for role in self.config_role:
            self.role_prompts[role] = "\n".join(self.config_role[role])

        # init log
        self.start_time, self.log_filepath = self.get_logfilepath()

  def make_recruitment(self):
    for employee in self.recruitments:
      self.chat_env.recruit(agent_name=employee)

  def execute_step(self, phase_item: dict):
    # Retrieve the single phase configuration
    phase = self.config_phase["phase"]
    phase_type = self.config_phase["phaseType"]
    max_turn_step = self.config_phase.get("max_turn_step", 10)
    need_reflect = check_bool(self.config_phase.get("need_reflect", "False"))

    # Check the phase type and execute accordingly
    if phase_type == "SimplePhase":
        if phase in self.phases:
            self.chat_env = self.phases[phase].execute(
                self.chat_env,
                max_turn_step,
                need_reflect
            )
        else:
            raise RuntimeError(f"Phase '{phase}' is not yet implemented in chatdev.phase")
    else:
        raise RuntimeError(f"Unsupported PhaseType '{phase_type}' in single-phase execution.")


  def execute_chain(self):
    for phase_item in self.chain:
      self.execute_step(phase_item)


  def self_task_improve(self, task_prompt):
    self_task_improve_prompt = """I will give you a short description of a software design requirement,
please rewrite it into a detailed prompt that can make large language model know how to make this software better based this prompt,
the prompt should ensure LLMs build a software that can be run correctly, which is the most import part you need to consider.
remember that the revised prompt should not contain more than 200 words,
here is the short description:\"{}\".
If the revised prompt is revised_version_of_the_description,
then you should return a message in a format like \"<INFO> revised_version_of_the_description\", do not return messages in other formats.""".format(
            task_prompt)
     role_play_session = RolePlaying(
         assistant_role_name="Prompt Engineer",
         assistant_role_prompt="You are an professional prompt engineer that can improve user input prompt to make LLM better understand these prompts.",
         user_role_prompt="You are an user that want to use LLM to build software.",
         user_role_name="User",
         task_type=TaskType.CHATDEV,
         task_prompt="Do prompt engineering on user query",
         with_task_specify=False,
         model_type=self.model_type,
        )

        # log_visualize("System", role_play_session.assistant_sys_msg)
        # log_visualize("System", role_play_session.user_sys_msg)

        _, input_user_msg = role_play_session.init_chat(None, None, self_task_improve_prompt)
        assistant_response, user_response = role_play_session.step(input_user_msg, True)
        revised_task_prompt = assistant_response.msg.content.split("<INFO>")[-1].lower().strip()
        log_visualize(role_play_session.assistant_agent.role_name, assistant_response.msg.content)
        log_visualize(
            "**[Task Prompt Self Improvement]**\n**Original Task Prompt**: {}\n**Improved Task Prompt**: {}".format(
                task_prompt, revised_task_prompt))
        return revised_task_promp

